In [8]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import numpy as np
import math,time
from datetime import timedelta

In [9]:
#权重
def new_W(shape):
    W=tf.Variable(tf.truncated_normal(shape,stddev=0.05))
    return W

#偏置
def new_b(length):
    b=tf.Variable(tf.constant(0.1,shape=length))
    return b

#卷积
def conv2d(x,W):
    conv=tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
    return conv

#最大池化
def max_pool_2x2(inputx):
    max_pool=tf.nn.max_pool(inputx,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    return max_pool

In [10]:
#加载数据集
data=input_data.read_data_sets('dataset/',one_hot=True)

#打印一些数据集信息
print('\n' 'train set:label_num={},img_num={}'.format(len(data.train.labels),data.train.num_examples))
print('test set:label_num={},img_num={}'.format(len(data.test.labels),data.train.num_examples))
print('validation set:label_num={},img_num={}'.format(len(data.validation.labels),data.validation.num_examples))

Extracting dataset/train-images-idx3-ubyte.gz
Extracting dataset/train-labels-idx1-ubyte.gz
Extracting dataset/t10k-images-idx3-ubyte.gz
Extracting dataset/t10k-labels-idx1-ubyte.gz

train set:label_num=55000,img_num=55000
test set:label_num=10000,img_num=55000
validation set:label_num=5000,img_num=5000


In [11]:
data.test.cls=np.argmax(data.test.labels,axis=1)
#输入值
x=tf.placeholder('float',shape=[None,784],name='x')
x_image=tf.reshape(x,[-1,28,28,1])

#输出值
y_true=tf.placeholder('float',shape=[None,10],name='y_true')
y_true_cls=tf.argmax(y_true,dimension=1)

In [18]:
#conv1 & relu & max_pool
layer_conv1={'weights':new_W([5,5,1,32]),'biases':new_b([32])}
h_conv1=tf.nn.relu(conv2d(x_image,layer_conv1['weights'])+layer_conv1['biases'])
h_pool1=max_pool_2x2(h_conv1)

#conv2 & relu & max_pool
layer_conv2={'weights':new_W([5,5,32,64]),'biases':new_b([64])}
h_conv2=tf.nn.relu(conv2d(h_pool1,layer_conv2['weights'])+layer_conv2['biases'])
h_pool2=max_pool_2x2(h_conv2)

#fc1-->flat-->relu
fc1_layer={'weights':new_W([7*7*64,1024]),'biases':new_b([1024])}
h_pool2_plat=tf.reshape(h_pool2,[-1,7*7*64])
h_fc1=tf.nn.relu(tf.matmul(h_pool2_plat,fc1_layer['weights'])+fc1_layer['biases'])

#Dropout
keep_prob=tf.placeholder('float')
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)

#fc2
fc2_layer={'weights':new_W([1024,10]),'biases':new_b([10])}

#使用sotfmax()做预测
y_pred=tf.nn.softmax(tf.matmul(h_fc1_drop,fc2_layer['weights'])+fc2_layer['biases'])
y_pred_cls=tf.argmax(y_pred,dimension=1) #显示真实预测数字

#使用RMSPropOptimizer()优化交叉熵损失函数
cross_entropy=-tf.reduce_mean(y_true * tf.log(y_pred))
# cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))
optimizer=tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(cross_entropy)

#计算准确率
correct_prediction=tf.equal(y_pred_cls,y_true_cls)
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [24]:
# 训练 & 测试
with tf.Session() as sess:
    init=tf.global_variables_initializer()
    sess.run(init)
    
    #训练
    train_batch_size=100
    display_num=500
    def train(train_epoch):
        total_iter=0
        start_time=time.time()
        
        for i in range(total_iter,total_iter+train_epoch):
            x_batch,y_true_batch=data.train.next_batch(train_batch_size)
            feed_dict_train_op={x:x_batch,y_true:y_true_batch,keep_prob:0.8}
            feed_dict_train={x:x_batch,y_true:y_true_batch,keep_prob:1.0}
            
            sess.run(optimizer,feed_dict=feed_dict_train_op)
            
            #训练状态
            if (i % display_num==0):
                #计算训练集的准确率
                acc=sess.run(accuracy,feed_dict=feed_dict_train)
                #打印消息
                msg='Iter:{0:>6},train_acc:{1:6.2%}'
                print(msg.format(i,acc))
        
#         total_iter+=train_epoch
        
        end_time=time.time()
        use_time=end_time-start_time
        print('time:'+str(timedelta(seconds=int(round(use_time)))))
    
    #测试
    test_batch_size=100
    def test():
        num_test=len(data.test.images)
        cls_pred=np.zeros(shape=num_test,dtype=np.int)
        
        i=0
        while i<num_test:
            j=min(i+test_batch_size,num_test)
            img=data.test.images[i:j,:]
            labels=data.test.labels[i:j,:]
            feed_dict={x:img,y_true:labels,keep_prob:1.0}
            cls_pred[i:j]=sess.run(y_pred_cls,feed_dict=feed_dict)
            i=j
            
        cls_true=data.test.cls
        correct=(cls_true==cls_pred)
        correct_sum=correct.sum()
        acc=float(correct_sum)/num_test
        
        msg='test_acc:{0:.2%} ({1}/{2})'
        print(msg.format(acc,correct_sum,num_test))

    
    train_epoch=4001
    train(train_epoch)
    test()

Iter:     0,train_acc: 5.00%
Iter:   500,train_acc:100.00%
Iter:  1000,train_acc:100.00%
Iter:  1500,train_acc:100.00%
Iter:  2000,train_acc:100.00%
Iter:  2500,train_acc:100.00%
Iter:  3000,train_acc:100.00%
Iter:  3500,train_acc:100.00%
Iter:  4000,train_acc:100.00%
time:0:00:17
test_acc:99.34% (9934/10000)


In [22]:
#更新
'''
激活函数：ReLU，Leaky-ReLU
卷积核3*3、1*1
优化器：GradientDescentOptimizer(0.01)，AdadeltaOptimizer(0.01)，AdamOptimizer()，RMSPropOptimizer(learning_rate=0.001)，AdagradOptimizer(learning_rate=0.01)等
学习率、dropout值
'''

'\n激活函数：Leaky-ReLU\n卷积核3*3、1*1\n优化器\n学习率、dropout值'